# Imports and constants

In [140]:
#import statemets

import numpy as np
from sklearn.neural_network import MLPClassifier
from customScripts import utilities as util
from customScripts import features as feat

In [141]:
# parameters

#number of samples / second
sampling_rate = 48000

#length of frame in samples
frame_length = 48000

#number of samples used as offset for earch consecutive frame
hop_length = 24000

#number of mel frequency bins to use
bin_number = 40 # also try 80

#set the desired number of frames / second here
ground_thruth_conversion_const = 2

#input data
file_paths = ['audio_files/Muppets-02-01-01.wav', 'audio_files/Muppets-02-04-04.wav', 'audio_files/Muppets-03-04-03.wav']
grount_truth_paths = ['ground_truth/Muppets-02-01-01.csv', 'ground_truth/Muppets-02-04-04.csv', 'ground_truth/Muppets-03-04-03.csv']
prediction_paths = ['predictions/Muppets-02-01-01.csv', 'predictions/Muppets-02-04-04.csv', 'predictions/Muppets-03-04-03.csv']
file_lengths = [1547, 1548, 1539] #in seconds

# File loading and feature extraction

In [142]:
#load episode 1
y_1 = util.load_audio(file_paths[0], sampling_rate)

In [143]:
#load episode 2
y_2 = util.load_audio(file_paths[1], sampling_rate)

In [144]:
#load episode 3
y_3 = util.load_audio(file_paths[2], sampling_rate)

In [145]:
#compute spectrogram for episode 1
spectrogram_1 = feat.compute_spectrogram(y_1, frame_length, hop_length, bin_number)
spectrogram_1T = spectrogram_1.transpose()

In [146]:
#compute spectrogram for episode 2
spectrogram_2 = feat.compute_spectrogram(y_2, frame_length, hop_length, bin_number)
spectrogram_2T = spectrogram_2.transpose()

In [147]:
#compute spectrogram for episode 3
spectrogram_3 = feat.compute_spectrogram(y_3, frame_length, hop_length, bin_number)
spectrogram_3T = spectrogram_3.transpose()

In [203]:
#load ground truth, pad it with zeroes and adjust to frame number

#episode 1
gt_1 = util.load_ground_truth(grount_truth_paths[0], '/n')
gt_1 = util.compute_0_padded_gt(gt_1, file_lengths[0])
gt_1 = util.adjust_gt_to_frames(gt_1, ground_thruth_conversion_const)
gt_1.append(0) # append a zero so that lenght equals the frames list length

#episode 2
gt_2 = util.load_ground_truth(grount_truth_paths[1], '/n')
gt_2 = util.compute_0_padded_gt(gt_2, file_lengths[1])
gt_2 = util.adjust_gt_to_frames(gt_2, ground_thruth_conversion_const)
gt_2.append(0) # append a zero so that lenght equals the frames list length

#episode 3
gt_3 = util.load_ground_truth(grount_truth_paths[2], '/n')
gt_3 = util.compute_0_padded_gt(gt_3, file_lengths[2])
gt_3 = util.adjust_gt_to_frames(gt_3, ground_thruth_conversion_const)
gt_3.append(0) # append a zero so that lenght equals the frames list length
gt_3.append(0) # append a zero so that lenght equals the frames list length

In [207]:
#concatenate training data

gt_train = gt_1 + gt_2

spectrogram_train = np.concatenate((spectrogram_1T, spectrogram_2T), axis=0)

spectrogram_predict = spectrogram_3T

#spectrogram_train[3195] == spectrogram_2T[100]

#for i,g in enumerate(gt_train):
#    if g == 1:
#        print(i)

# Classifier Training and prediction

In [214]:
clf = MLPClassifier()

In [215]:
clf.fit(spectrogram_train, gt_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [217]:
prediction = clf.predict(spectrogram_predict)

In [219]:
np.savetxt(prediction_paths[2], prediction, delimiter="/n")